In [2]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

data_path = r'e:\projects\Python\FindingElo\data\Lichess\stockfish\\'
all_features_path = data_path+'all_features\\all_features.pickle'

In [4]:
# Считываем вычисленные характеристики
X = pd.read_pickle(all_features_path)
X

,0,1,2,3,4,5,6,7,8,9,...,2273,2274,2275,2276,2277,2278,2279,2280,2281,2282
0,5.0,0.0,17.0,0.0,0.0,1.0,1.0,7.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
1,11.0,0.0,12.0,0.0,0.0,1.0,2.0,6.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,7.0,30.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,10.0,0.0,1.0,1.0,0.0,4.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0
4,8.0,0.0,15.0,0.0,1.0,1.0,1.0,4.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27796,9.0,0.0,12.0,0.0,2.0,1.0,0.0,4.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
27797,0.0,0.0,4.0,5.0,2.0,1.0,1.0,7.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
27798,0.0,0.0,6.0,15.0,0.0,1.0,0.0,3.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
27799,0.0,0.0,6.0,27.0,1.0,1.0,0.0,0.0,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
with open(data_path+'data.pickle', 'rb') as f:
    data = pd.read_pickle(f)
data

,ELO_white,ELO_black,Result,ECO,Opening,UTCDate,TimeControl,Termination
0,2126.0,2576.0,1.0,A40,English Defense #2,2013-10-15,bullet,Normal
1,2526.0,2163.0,1.0,A00,Van't Kruijs Opening,2013-10-15,bullet,Normal
2,1767.0,2508.0,1.0,B21,Sicilian Defense: McDonnell Attack,2013-10-17,blitz,Normal
3,2558.0,2006.0,1.0,B14,Caro-Kann Defense: Panov Attack,2013-10-20,bullet,Time forfeit
4,1762.0,2480.0,1.0,B78,"Sicilian Defense: Dragon Variation, Yugoslav A...",2013-10-21,bullet,Normal
...,...,...,...,...,...,...,...,...
27796,2456.0,2304.0,1.0,B07,Rat Defense: Antal Defense,2015-09-30,bullet,Normal
27797,2307.0,2526.0,0.0,B07,Pirc Defense #4,2015-09-30,bullet,Time forfeit
27798,2306.0,2454.0,1.0,A04,Reti Opening #2,2015-09-30,bullet,Time forfeit
27799,2533.0,2301.0,1.0,A00,Hungarian Opening,2015-09-30,bullet,Normal


In [117]:
X = X[:2000]
data = data[:2000]

In [118]:
mean_elos = (data.ELO_white + data.ELO_black) / 2
elo_difference = data.ELO_white - data.ELO_black

In [124]:
X_train_mean, X_test_mean, Y_train_mean, Y_test_mean = train_test_split(X, mean_elos, test_size=0.1, random_state=0)
mean_model = keras.Sequential()
mean_model.add(keras.layers.Dense(X_train_mean.shape[0], activation=tf.nn.relu))
mean_model.add(keras.layers.Dense(512, activation=tf.nn.relu))
mean_model.add(keras.layers.Dense(74, activation=tf.nn.relu))
mean_model.add(keras.layers.Dense(1, activation='linear'))
mean_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='mean_squared_error',
              metrics=['accuracy'])
mean_model.fit(X_train_mean, Y_train_mean, epochs=140)

Epoch 1/20
8/8 [==============================] - 1s 49ms/step - loss: 3116455.5000 - accuracy: 0.0000e+00
Epoch 2/20
8/8 [==============================] - 0s 47ms/step - loss: 1408015.1250 - accuracy: 0.0000e+00
Epoch 3/20
8/8 [==============================] - 0s 47ms/step - loss: 787849.2500 - accuracy: 0.0000e+00
Epoch 4/20
8/8 [==============================] - 0s 48ms/step - loss: 433922.2500 - accuracy: 0.0000e+00
Epoch 5/20
8/8 [==============================] - 0s 47ms/step - loss: 325363.1875 - accuracy: 0.0000e+00
Epoch 6/20
8/8 [==============================] - 0s 48ms/step - loss: 271467.1562 - accuracy: 0.0000e+00
Epoch 7/20
8/8 [==============================] - 0s 55ms/step - loss: 229559.7812 - accuracy: 0.0000e+00
Epoch 8/20
8/8 [==============================] - 0s 48ms/step - loss: 183302.2812 - accuracy: 0.0000e+00
Epoch 9/20
8/8 [==============================] - 0s 49ms/step - loss: 147588.0625 - accuracy: 0.0000e+00
Epoch 10/20
8/8 [===========================

In [125]:
X_train_diff, X_test_diff, Y_train_diff, Y_test_diff = train_test_split(X, elo_difference, test_size=0.1, random_state=0)
diff_model = keras.Sequential()
diff_model.add(keras.layers.Dense(X_train_diff.shape[0], activation=tf.nn.relu))
diff_model.add(keras.layers.Dense(512, activation=tf.nn.relu))
diff_model.add(keras.layers.Dense(74, activation=tf.nn.relu))
diff_model.add(keras.layers.Dense(1, activation='linear'))
diff_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='mean_squared_error',
              metrics=['accuracy'])
diff_model.fit(X_train_diff, Y_train_diff, epochs=140)

Epoch 1/20
57/57 [==============================] - 2s 28ms/step - loss: 115675.1172 - accuracy: 5.5556e-04
Epoch 2/20
57/57 [==============================] - 2s 29ms/step - loss: 113181.3984 - accuracy: 5.5556e-04
Epoch 3/20
57/57 [==============================] - 2s 29ms/step - loss: 111861.2734 - accuracy: 0.0011
Epoch 4/20
57/57 [==============================] - 2s 29ms/step - loss: 110737.3672 - accuracy: 0.0017
Epoch 5/20
57/57 [==============================] - 2s 29ms/step - loss: 111048.8516 - accuracy: 5.5556e-04
Epoch 6/20
57/57 [==============================] - 2s 29ms/step - loss: 109416.9141 - accuracy: 0.0011
Epoch 7/20
57/57 [==============================] - 2s 29ms/step - loss: 106599.6328 - accuracy: 0.0011
Epoch 8/20
57/57 [==============================] - 2s 29ms/step - loss: 103571.0938 - accuracy: 5.5556e-04
Epoch 9/20
57/57 [==============================] - 2s 29ms/step - loss: 103137.7344 - accuracy: 0.0017
Epoch 10/20
57/57 [=============================

In [126]:
test_loss, test_acc = mean_model.evaluate(X_test_mean, Y_test_mean)
print('Test accuracy:', test_acc)

7/7 [==============================] - 0s 7ms/step - loss: 67850.5625 - accuracy: 0.0000e+00
Test accuracy: 0.0


In [127]:
test_loss, test_acc = diff_model.evaluate(X_test_diff, Y_test_diff)
print('Test accuracy:', test_acc)

7/7 [==============================] - 0s 13ms/step - loss: 68428.5156 - accuracy: 0.0000e+00
Test accuracy: 0.0


In [134]:
# Предсказание на тех же данных, на которых обучались
mean_pred_rf = np.array(mean_model.predict(X_test_mean))
elo_difference_pred = np.array(diff_model.predict(X_test_diff))
white_elos = mean_pred_rf + elo_difference_pred / 2
black_elos = mean_pred_rf - elo_difference_pred / 2
wdiffs = np.sum((data.ELO_white[-len(white_elos):].values - white_elos)**2)
bdiffs = np.sum((data.ELO_black[-len(black_elos):].values - black_elos)**2)
print(wdiffs, bdiffs, wdiffs+bdiffs)

7/7 [==============================] - 0s 7ms/step
3952631491.622808 4098388468.7615924 8051019960.3844


In [132]:
predictions = mean_model.predict(X_test_mean)
predictions

7/7 [==============================] - 0s 7ms/step


array([[2047.0369],
       [2624.0947],
       [2358.8018],
       [1947.7255],
       [2494.1355],
       [2611.17  ],
       [2506.9097],
       [2398.2246],
       [2229.9817],
       [2016.5529],
       [2554.775 ],
       [2631.064 ],
       [2012.5704],
       [2482.5564],
       [2124.3296],
       [2741.2876],
       [2010.9426],
       [2633.1423],
       [2614.9314],
       [2061.675 ],
       [2598.7588],
       [2405.439 ],
       [2302.481 ],
       [2441.497 ],
       [2507.0085],
       [2552.6865],
       [1893.4644],
       [2296.493 ],
       [2369.9478],
       [2620.8123],
       [2761.1873],
       [2305.439 ],
       [2709.035 ],
       [2730.3608],
       [2669.2532],
       [2367.3413],
       [2573.928 ],
       [2496.2432],
       [2200.6902],
       [2422.0032],
       [2340.2961],
       [2653.005 ],
       [2571.439 ],
       [2165.0352],
       [2804.0452],
       [2533.748 ],
       [1693.6266],
       [2518.8555],
       [2681.8406],
       [2287.9175],


In [133]:
list(zip(white_elos, black_elos))

[(array([1947.6938], dtype=float32), array([2146.38], dtype=float32)),
 (array([2409.551], dtype=float32), array([2838.6384], dtype=float32)),
 (array([2257.5747], dtype=float32), array([2460.0288], dtype=float32)),
 (array([1926.4277], dtype=float32), array([1969.0232], dtype=float32)),
 (array([2550.833], dtype=float32), array([2437.438], dtype=float32)),
 (array([2495.1248], dtype=float32), array([2727.215], dtype=float32)),
 (array([2608.5447], dtype=float32), array([2405.2747], dtype=float32)),
 (array([2299.1978], dtype=float32), array([2497.2515], dtype=float32)),
 (array([2170.8538], dtype=float32), array([2289.1096], dtype=float32)),
 (array([1944.3895], dtype=float32), array([2088.7163], dtype=float32)),
 (array([2497.7703], dtype=float32), array([2611.7795], dtype=float32)),
 (array([2558.9543], dtype=float32), array([2703.1736], dtype=float32)),
 (array([1993.5834], dtype=float32), array([2031.5575], dtype=float32)),
 (array([2444.5647], dtype=float32), array([2520.548], dt